AHP map matching algorithm for envirocar

In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
from IMM import IMM
from Junction_Condition import JC
from MMJ import MMJ

# import sys
# sys.path.append('..')

In [3]:
def AHPMM(trajectory_data, nodes_data, edges_data):
    matched_links = []
    matched_links_no_duplicate = []
    # matched_points = []
    iter, previous_edge = IMM(trajectory_data, edges_data, iter=0)
    previous_edge['method'] = ['IMM']
    matched_links.append(previous_edge)
    matched_links_no_duplicate.append(previous_edge)
    while iter < len(trajectory_data):
        current_position = trajectory_data.iloc[[iter]]
        previous_position = trajectory_data.iloc[[iter-1]]
        # print(current_position)
        if JC(nodes_data, edges_data, current_position, previous_position, previous_edge):
            previous_edge['method'] = ['JC']
            matched_links.append(previous_edge)
            iter = iter + 1
        else:
            temp = previous_edge
            iter, previous_edge = MMJ(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter)
            previous_edge['method'] = ['SMM']
            matched_links.append(previous_edge)
            if (previous_edge.index[0] != temp.index[0]) and (previous_edge.iloc[0].method == 'SMM'):
                matched_links_no_duplicate.append(previous_edge)

    return matched_links, matched_links_no_duplicate

In [20]:
import sys
sys.path.append('..')
import preprocessing.downloader as dl
id = dl.fetch_ids_on_envirocar(num_trajectories=5)[4]
gdf = dl.download_from_envirocar(id, use_cache=False)
trajectory_data, nodes_data, edges_data = dl.to_fuzzy_AHP_input(gdf)

In [21]:
matched_links, matched_links_no_duplicate = AHPMM(trajectory_data, nodes_data, edges_data)

In [22]:
matched_edge_df = pd.concat(matched_links)
# matched_points_df = pd.concat(matched_points)

In [23]:
# visualization
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point

point_locs = trajectory_data['geometry'].to_frame()
point_locs.plot(ax = ax, color = "Red")


# this plot all the road system 
edges_data.plot(ax=ax)


matched_edge_df.plot(ax = ax, color = "Orange")


<Axes: >

In [7]:
len(trajectory_data)

463